In [ ]:
# Program - 3
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_d(x):
    return x * (1 - x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.w_i_h = np.random.randn(input_size, hidden_size)
        self.b_h = np.random.randn(1, hidden_size)
        self.w_h_o = np.random.randn(hidden_size, output_size)
        self.b_o = np.random.randn(1, output_size)

    def forward(self, x):
        self.h_a = np.dot(x, self.w_i_h) + self.b_h
        self.h_o = sigmoid(self.h_a)

        self.o_a = np.dot(self.h_o, self.w_h_o) + self.b_o
        self.o_o = softmax(self.o_a)

        return self.o_o

    def backward(self, x, y, output, lr):
        m = y.shape[0]
        d_o = output - y
        e_h = d_o.dot(self.w_h_o.T)
        d_h = e_h * sigmoid_d(self.h_o)

        self.w_h_o -= self.h_o.T.dot(d_o) * lr / m
        self.b_o -= np.sum(d_o, axis=0, keepdims=True) * lr /m
        self.w_i_h -= x.T.dot(d_h) *lr / m
        self.b_h -= np.sum(d_h, axis=0, keepdims=True) * lr /m

    def loss_calculation(self, y_true, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
        loss = -np.mean(np.sum(y_true * np.log(y_pred), axis=1))
        return loss


x = np.array([[0,0], [1,0], [0,1], [1,1]])
y = np.array([[1,0,0], [0,1,0], [0,1,0], [0,0,1]])

input_size, hidden_size, output_size = 2, 4, 3
lr = 0.1
epochs = 10000

nn = NeuralNetwork(input_size, hidden_size, output_size)

# train
for i in range(epochs):
    output = nn.forward(x)
    nn.backward(x, y, output, lr)

    if i % 1000 == 0:
        loss = nn.loss_calculation(y, output)
        print(f"Epoch {i} | Loss {loss:.4f}")

print("predicting outputs ")
prediction = nn.forward(x)
print(prediction)

print("\npredicting classes ")
print(np.argmax(prediction, axis=1))